<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/brain_tumor_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install important libraries
!pip install tensorflow-model-optimization
!pip install opendatasets

In [ ]:
# Import necessary libraries and functions
import opendatasets as od
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import tensorflow_model_optimization
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude,PolynomialDecay,strip_pruning

In [ ]:
# load the data into working directory
od.download("https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri")


In [4]:
# Set the ImageDataGenerator for data augmentation
datagen=ImageDataGenerator(rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
# Create train and test generators
train_generator=datagen.flow_from_directory("/content/brain-tumor-classification-mri/Training",class_mode="categorical",batch_size=64)
test_generator=datagen.flow_from_directory("/content/brain-tumor-classification-mri/Testing",class_mode="categorical",batch_size=64)

In [13]:
# Create an EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True)

In [43]:
# Create a model
def create_model():
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(90, activation='relu'))
    model.add(Dense(4, activation='softmax'))
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [ ]:
# Check model's summary
model=create_model()
model.summary()

In [ ]:
# Train your model with callbacks
model.fit(train_generator, validation_data=test_generator,epochs=15,batch_size=32,callbacks=[early_stopping],verbose=1)

In [ ]:
# Evaluate the model(with best weights) over test_generator
model.evaluate(test_generator)

In [39]:
# Set the pruning parameters
num_epochs = 12
begin_epoch = 0
end_epoch = num_epochs
pruning_params = {
    'pruning_schedule': PolynomialDecay(initial_sparsity=0.2, final_sparsity=0.8, begin_step=begin_epoch, end_step=end_epoch, frequency=10)
}

# Apply pruning to the original model using the specified pruning parameters
pruned_model = prune_low_magnitude(model, **pruning_params)

# Compile the pruned model with the same settings as the original model
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Evaluate the pruned_model over testing data
pruned_model.evaluate(test_generator)

In [ ]:
# Check the summary of the pruned model
pruned_model.summary()

In [ ]:
# Save pruned model as a file
pruned_model.save("my_model.h5")